# HW3

By Group C

In [2]:
import open3d as o3d
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation
from sklearn.metrics import mean_squared_error

import pyrosbag as prb
from rosbags.highlevel import AnyReader
import os
from pathlib import Path
import rosbag
from cv_bridge import CvBridge

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2023-06-13 12:36:27,962 - _logger - Log opened: Tue Jun 13 10:36:27 2023 UTC
INFO - 2023-06-13 12:36:27,977 - topics - topicmanager initialized


In [2]:
!source /opt/ros/noetic/setup.bash

In [3]:
%%capture
!pip install bagpy

# Task 1

### Task 1.1

In [3]:
# read source point cloud
pcd_source = o3d.io.read_point_cloud('assets/models/oats/texturedMesh_alligned_vertex_color.ply')

### Task 1.2

In [4]:

voxel_size = 0.01
pcd_source = pcd_source.voxel_down_sample(voxel_size)

### Output

In [5]:
o3d.visualization.draw_geometries([pcd_source])

# Task 2
### Task 2.1 and 2.2

In [3]:
#bag_file = "bag_2022-05-11-14-37-56_0.bag"
bag_file = "icp_tracking_oats.bag"
#camera_topic = "/camera/141722071427/"
camera_topic = "/camera/"
depth_info = "aligned_depth_to_color/camera_info"
depth_raw = "aligned_depth_to_color/image_raw"
color_info = "color/camera_info"
color_raw = "color/image_raw"

In [4]:
########### For first fifty frames ###############################

In [ ]:
import open3d as o3d

bag = rosbag.Bag(bag_file)
frame = 0
last_color = None
last_depth = None
pcd_list = []  # List to store PointCloud objects

for topic, msg, ts in bag:
    if frame >= 50:  # Break the loop after capturing 50 frames
        break

    if topic == camera_topic + color_raw:
        image_data = msg
        im = np.frombuffer(image_data.data, dtype=np.uint8).reshape(image_data.height, image_data.width, -1)
        color = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        color = np.flip(color, axis=1)
        last_color = color
    elif topic == camera_topic + depth_raw:
        image_data = msg
        im = np.frombuffer(image_data.data, dtype=np.uint16).reshape(image_data.height, image_data.width, -1)
        depth = np.flip(im, axis=1)
        last_depth = depth
    elif topic == camera_topic + color_info:
        last_msg = msg

    if last_color is not None and last_depth is not None:
        rgb_image = o3d.geometry.Image(last_color)
        depth_image = o3d.geometry.Image(last_depth)

        rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
            rgb_image, depth_image, convert_rgb_to_intensity=False)

        fx = last_msg.K[0]
        fy = last_msg.K[4]
        cx = last_msg.K[2]
        cy = last_msg.K[5]
        width = last_msg.width
        height = last_msg.height

        camera_model = o3d.camera.PinholeCameraIntrinsic(width=width, height=height, fx=fx, fy=fy, cx=cx, cy=cy)

        P_inv = np.array(last_msg.P).reshape((3, 4))
        P_inv = np.vstack((P_inv, np.array([0, 0, 0, 1])))
        P_inv = np.linalg.inv(P_inv)

        pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, camera_model, P_inv)

        pcd_list.append(pcd)
        frame += 1

bag.close()

merged_pcd = o3d.geometry.PointCloud()
for pcd in pcd_list:
    merged_pcd += pcd

o3d.visualization.draw_geometries([merged_pcd])

# Task 3

### Task 3.1 and 3.2

In [ ]:
import copy

pcd = merged_pcd
o3d.visualization.draw_geometries_with_editing([pcd])

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
    
def pick_points(pcd):
    print("")
    print("1) Please pick at least three correspondences using [shift + left click]")
    print("   Press [shift + right click] to undo point picking")
    print("2) After picking points, press 'Q' to close the window")
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window()
    vis.add_geometry(pcd)
    vis.run()  # User picks points
    vis.destroy_window()
    print("")
    return vis.get_picked_points()

def demo_manual_registration(pcd_arg, oats_can_path):
    print("Demo for manual ICP")
    
    print("Generating first window (Textured Mesh Visualisation)")
    oats_can_pcd = o3d.io.read_point_cloud(oats_can_path)

    print("Generating second window (Registration Result before Correspondences)")
    draw_registration_result(pcd_arg, oats_can_pcd, np.identity(4))

    print("Generating third window (Picking points from Point Cloud of Textured Mesh)")
    picked_id_source = pick_points(pcd_arg)
    
    print("Generating fourth window (Picking points from oats can)")
    picked_id_target = pick_points(oats_can_pcd)

    assert len(picked_id_source) >= 3 and len(picked_id_target) >= 3
    assert len(picked_id_source) == len(picked_id_target)

    corr = np.zeros((len(picked_id_source), 2))
    corr[:, 0] = picked_id_source
    corr[:, 1] = picked_id_target

    print("Compute a rough transform using the correspondences given by the user")
    p2p = o3d.pipelines.registration.TransformationEstimationPointToPoint()
    trans_init = p2p.compute_transformation(pcd_arg, oats_can_pcd, o3d.utility.Vector2iVector(corr))

    print("Perform point-to-point ICP refinement")
    threshold = 0.03 
    reg_p2p = o3d.pipelines.registration.registration_icp(
        pcd_arg, oats_can_pcd, threshold, trans_init, p2p)

    print("Generating fifth window (Registration Result After Correspondences)")
    draw_registration_result(pcd_arg, oats_can_pcd, reg_p2p.transformation)
    print("")
    
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
    
demo_manual_registration(pcd, "assets/models/oats/texturedMesh_alligned_vertex_color.ply")

# Task 4

In [ ]:
import open3d as o3d

bag = rosbag.Bag(bag_file)
frame = 0
last_color = None
last_depth = None
pcd_list = []  

for topic, msg, ts in bag:
    if frame >= 50: 
        break

    if topic == camera_topic + color_raw:

        image_data = msg
        im = np.frombuffer(image_data.data, dtype=np.uint8).reshape(image_data.height, image_data.width, -1)
        color = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

        color = np.flip(color, axis=1)
        last_color = color
    elif topic == camera_topic + depth_raw:

        image_data = msg
        im = np.frombuffer(image_data.data, dtype=np.uint16).reshape(image_data.height, image_data.width, -1)

        depth = np.flip(im, axis=1)
        last_depth = depth
    elif topic == camera_topic + color_info:
        last_msg = msg

    if last_color is not None and last_depth is not None:

        rgb_image = o3d.geometry.Image(last_color)
        depth_image = o3d.geometry.Image(last_depth)

        rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
            rgb_image, depth_image, convert_rgb_to_intensity=False)


        fx = last_msg.K[0]
        fy = last_msg.K[4]
        cx = last_msg.K[2]
        cy = last_msg.K[5]
        width = last_msg.width
        height = last_msg.height

        camera_model = o3d.camera.PinholeCameraIntrinsic(width=width, height=height, fx=fx, fy=fy, cx=cx, cy=cy)

        P_inv = np.array(last_msg.P).reshape((3, 4))
        P_inv = np.vstack((P_inv, np.array([0, 0, 0, 1])))
        P_inv = np.linalg.inv(P_inv)


        pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, camera_model, P_inv)
        demo_manual_registration(pcd, "assets/models/oats/texturedMesh_alligned_vertex_color.ply")

#         # Append PointCloud to the list
#         pcd_list.append(pcd)
#         frame += 1

bag.close()

# # Merge the point clouds
# merged_pcd = o3d.geometry.PointCloud()
# for pcd in pcd_list:
#     merged_pcd += pcd

# # Visualize the merged point cloud
# o3d.visualization.draw_geometries([merged_pcd])

Demo for manual ICP
Generating first window (Textured Mesh Visualisation)
Generating second window (Registration Result before Correspondences)
Generating third window (Picking points from Point Cloud of Textured Mesh)

1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] Picked point #136873 (1.3e+02, 58., 0.21) to add in queue.
[Open3D INFO] Picked point #46073 (83., 26., 0.22) to add in queue.
[Open3D INFO] Picked point #102690 (18., 41., 0.18) to add in queue.
[Open3D INFO] Picked point #171622 (63., 70., 0.2) to add in queue.

Generating fourth window (Picking points from oats can)

1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] Picked point #64514 (-0.36, -0.28, -0.047) to add in queue.
[Open3D 